In [1]:
import pandas as pd
import os
from vllm import LLM, SamplingParams
import argparse
import gc
import glob
import os

os.environ['VLLM_USE_MODELSCOPE'] = "True"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
dashed_line = '-'*50

/home/amey/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-14 20:44:10,336	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [25]:
for filename in glob.glob('/home/amey/projects/multilingual-long-context-LLMs/prompts/*.pkl'):
    print(filename)

In [14]:
import random

def insert_sentences_randomly(distractor_passages, input_sentences):
    max_range = len(distractor_passages) + len(input_sentences)
    available_positions = list(range(max_range))
    positions = sorted(random.sample(available_positions, len(input_sentences)))

    result = []
    dp_index = 0
    is_index = 0

    for i in range(max_range):
        if is_index < len(input_sentences) and i == positions[is_index]:
            result.append(input_sentences[is_index])
            is_index += 1
        else:
            result.append(distractor_passages[dp_index])
            dp_index += 1

    return result

def insert_needle(
    input_passage: str, distractor_passages: list[str], distractor_type=str
) -> list[str]:
    if len(distractor_passages) <= 12:
        _ = []
        for x in distractor_passages:
            _.extend([x.strip() for x in x.split('.') if len(x) > 1])
        distractor_passages = _
         
    input_sentences = input_passage.split('.')
    input_sentences = [f"{sentence.strip()}." for sentence in input_sentences if len(sentence) > 1]
    input_sentences = insert_sentences_randomly(distractor_passages, input_sentences)
    if distractor_type == str:
        long_context = ' '.join(input_sentences)
    else:
      long_context = ''.join([sentence for sentence in input_sentences])
    return long_context

### Fewshot w Distractors

In [ ]:
from prompt_templates import (
    prompt_template_zs_qa1,
    prompt_template_zs_qa2,
    prompt_template_zs_qa3,
    prompt_template_zs_qa4,
    prompt_template_zs_qa5,
    prompt_template_zs_qa6,
    prompt_template_zs_qa7,
    prompt_template_zs_qa8,
    prompt_template_zs_qa9,
    prompt_template_zs_qa10
)

data_dir = '/home/amey/projects/multilingual-long-context-LLMs/data/input'
prompt_dir = '/home/amey/projects/multilingual-long-context-LLMs/prompts/zeroshot_random'

for num_distractors in [2, 4, 8, 10, 12]:
    for filename in glob.glob(f"{data_dir}/*.pkl"):
        df = pd.read_pickle(filename)
        
        # Take only 10 values per question
        df = df.groupby("question_type").sample(n=10, random_state=42)
        df = df.reset_index(drop=True)

        prompts = []
        for i, row in df.iterrows():
            question_type = row['question_type']
            input_passage = row['passage']
            exemplars = get_fewshot_examplers(input_passage, corpus, num_exemplars=4)
            distractor_passages = row['distractor_passages_mlqa'][:num_distractors]
            # distractor_passages = ["#"*len(x) for x in distractor_passages]
            # context = insert_needle(input_passage, distractor_passages, distractor_type=int)
            context = insert_needle(input_passage, distractor_passages)
            
            if question_type == 'qa1':
                prompt = prompt_template_zs_qa1(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa2':
                prompt = prompt_template_zs_qa2(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa3':
                prompt = prompt_template_zs_qa3(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa4':
                prompt = prompt_template_zs_qa4(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa5':
                prompt = prompt_template_zs_qa5(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa6':
                prompt = prompt_template_zs_qa6(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa7':
                prompt = prompt_template_zs_qa7(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa8':
                prompt = prompt_template_zs_qa8(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa9':
                prompt = prompt_template_zs_qa9(context=context, question=row['question'], exemplars=exemplars)
            elif question_type == 'qa10':
                prompt = prompt_template_zs_qa10(context=context, question=row['question'], exemplars=exemplars)
            prompts.append(prompt.strip())

        df['prompt'] = prompts
        lang = df.iloc[0].input_lang
        df[['id', 'input_lang', 'prompt', 'target']].to_pickle(f"{prompt_dir}/zs_{lang}_{num_distractors}.pkl")

### COT w Distractors

In [22]:
from prompt_templates import (
    prompt_template_cot_qa1,
    prompt_template_cot_qa2,
    prompt_template_cot_qa3,
    prompt_template_cot_qa4,
    prompt_template_cot_qa5,
    prompt_template_cot_qa6,
    prompt_template_cot_qa7,
    prompt_template_cot_qa8,
    prompt_template_cot_qa9,
    prompt_template_cot_qa10
)

data_dir = '/home/amey/projects/multilingual-long-context-LLMs/data/input'
prompt_dir = '/home/amey/projects/multilingual-long-context-LLMs/prompts/cot_random'

for num_distractors in [2, 4, 8, 10, 12]:
    for filename in glob.glob(f"{data_dir}/*.pkl"):
        df = pd.read_pickle(filename)
        
        # Take only 10 values per question
        df = df.groupby("question_type").sample(n=10, random_state=42)
        df = df.reset_index(drop=True)

        prompts = []
        for i, row in df.iterrows():
            question_type = row['question_type']
            input_passage = row['passage']
            distractor_passages = row['distractor_passages_synthetic'][:num_distractors]
            distractor_passages = ["#"*len(x) for x in distractor_passages]
            context = insert_needle(input_passage, distractor_passages, distractor_type=int)
            # context = insert_needle(input_passage, distractor_passages)
            
            if question_type == 'qa1':
                prompt = prompt_template_cot_qa1(context=context, question=row['question'])
            elif question_type == 'qa2':
                prompt = prompt_template_cot_qa2(context=context, question=row['question'])
            elif question_type == 'qa3':
                prompt = prompt_template_cot_qa3(context=context, question=row['question'])
            elif question_type == 'qa4':
                prompt = prompt_template_cot_qa4(context=context, question=row['question'])
            elif question_type == 'qa5':
                prompt = prompt_template_cot_qa5(context=context, question=row['question'])
            elif question_type == 'qa6':
                prompt = prompt_template_cot_qa6(context=context, question=row['question'])
            elif question_type == 'qa7':
                prompt = prompt_template_cot_qa7(context=context, question=row['question'])
            elif question_type == 'qa8':
                prompt = prompt_template_cot_qa8(context=context, question=row['question'])
            elif question_type == 'qa9':
                prompt = prompt_template_cot_qa9(context=context, question=row['question'])
            elif question_type == 'qa10':
                prompt = prompt_template_cot_qa10(context=context, question=row['question'])
            prompts.append(prompt.strip())

        df['prompt'] = prompts
        lang = df.iloc[0].input_lang
        df[['id', 'input_lang', 'prompt', 'target']].to_pickle(f"{prompt_dir}/zs_{lang}_{num_distractors}.pkl")


### Translation w Distractors

In [20]:
from prompt_templates import (
    prompt_template_zs_translation_qa1,
    prompt_template_zs_translation_qa2,
    prompt_template_zs_translation_qa3,
    prompt_template_zs_translation_qa4,
    prompt_template_zs_translation_qa5,
    prompt_template_zs_translation_qa6,
    prompt_template_zs_translation_qa7,
    prompt_template_zs_translation_qa8,
    prompt_template_zs_translation_qa9,
    prompt_template_zs_translation_qa10
)

data_dir = '/home/amey/projects/multilingual-long-context-LLMs/data/input'
prompt_dir = '/home/amey/projects/multilingual-long-context-LLMs/prompts/translation_synthetic'

for num_distractors in [2, 4, 8, 10, 12]:
    for filename in glob.glob(f"{data_dir}/*.pkl"):
        df = pd.read_pickle(filename)
        
        # Take only 10 values per question
        df = df.groupby("question_type").sample(n=10, random_state=42)
        df = df.reset_index(drop=True)

        prompts = []
        for i, row in df.iterrows():
            question_type = row['question_type']
            input_passage = row['passage']
            distractor_passages = row['distractor_passages_synthetic'][:num_distractors]
            # distractor_passages = ["#"*len(x) for x in distractor_passages]
            # context = insert_needle(input_passage, distractor_passages, distractor_type=int)
            context = insert_needle(input_passage, distractor_passages)
            
            if question_type == 'qa1':
                prompt = prompt_template_zs_translation_qa1(context=context, question=row['question'])
            elif question_type == 'qa2':
                prompt = prompt_template_zs_translation_qa2(context=context, question=row['question'])
            elif question_type == 'qa3':
                prompt = prompt_template_zs_translation_qa3(context=context, question=row['question'])
            elif question_type == 'qa4':
                prompt = prompt_template_zs_translation_qa4(context=context, question=row['question'])
            elif question_type == 'qa5':
                prompt = prompt_template_zs_translation_qa5(context=context, question=row['question'])
            elif question_type == 'qa6':
                prompt = prompt_template_zs_translation_qa6(context=context, question=row['question'])
            elif question_type == 'qa7':
                prompt = prompt_template_zs_translation_qa7(context=context, question=row['question'])
            elif question_type == 'qa8':
                prompt = prompt_template_zs_translation_qa8(context=context, question=row['question'])
            elif question_type == 'qa9':
                prompt = prompt_template_zs_translation_qa9(context=context, question=row['question'])
            elif question_type == 'qa10':
                prompt = prompt_template_zs_translation_qa10(context=context, question=row['question'])
            prompts.append(prompt.strip())

        df['prompt'] = prompts
        lang = df.iloc[0].input_lang
        df[['id', 'input_lang', 'prompt', 'target']].to_pickle(f"{prompt_dir}/zs_{lang}_{num_distractors}.pkl")

### Zeroshot w Distractors

In [24]:
from prompt_templates import (
    prompt_template_zs_qa1,
    prompt_template_zs_qa2,
    prompt_template_zs_qa3,
    prompt_template_zs_qa4,
    prompt_template_zs_qa5,
    prompt_template_zs_qa6,
    prompt_template_zs_qa7,
    prompt_template_zs_qa8,
    prompt_template_zs_qa9,
    prompt_template_zs_qa10
)

data_dir = '/home/amey/projects/multilingual-long-context-LLMs/data/input'
prompt_dir = '/home/amey/projects/multilingual-long-context-LLMs/prompts/zeroshot_random'

for num_distractors in [2, 4, 8, 10, 12]:
    for filename in glob.glob(f"{data_dir}/*.pkl"):
        df = pd.read_pickle(filename)
        
        # Take only 10 values per question
        df = df.groupby("question_type").sample(n=10, random_state=42)
        df = df.reset_index(drop=True)

        prompts = []
        for i, row in df.iterrows():
            question_type = row['question_type']
            input_passage = row['passage']
            distractor_passages = row['distractor_passages_mlqa'][:num_distractors]
            distractor_passages = ["#"*len(x) for x in distractor_passages]
            context = insert_needle(input_passage, distractor_passages, distractor_type=int)
            # context = insert_needle(input_passage, distractor_passages)
            
            if question_type == 'qa1':
                prompt = prompt_template_zs_qa1(context=context, question=row['question'])
            elif question_type == 'qa2':
                prompt = prompt_template_zs_qa2(context=context, question=row['question'])
            elif question_type == 'qa3':
                prompt = prompt_template_zs_qa3(context=context, question=row['question'])
            elif question_type == 'qa4':
                prompt = prompt_template_zs_qa4(context=context, question=row['question'])
            elif question_type == 'qa5':
                prompt = prompt_template_zs_qa5(context=context, question=row['question'])
            elif question_type == 'qa6':
                prompt = prompt_template_zs_qa6(context=context, question=row['question'])
            elif question_type == 'qa7':
                prompt = prompt_template_zs_qa7(context=context, question=row['question'])
            elif question_type == 'qa8':
                prompt = prompt_template_zs_qa8(context=context, question=row['question'])
            elif question_type == 'qa9':
                prompt = prompt_template_zs_qa9(context=context, question=row['question'])
            elif question_type == 'qa10':
                prompt = prompt_template_zs_qa10(context=context, question=row['question'])
            prompts.append(prompt.strip())

        df['prompt'] = prompts
        lang = df.iloc[0].input_lang
        df[['id', 'input_lang', 'prompt', 'target']].to_pickle(f"{prompt_dir}/zs_{lang}_{num_distractors}.pkl")

### Setting up distractor passages for prompting

In [ ]:
# Load your data into DataFrames
source_df = pd.read_csv('/home/amey/projects/multilingual-long-context-LLMs/data/babilong/multilingual_bAbi/en.csv')[:10]
lang = source_df.iloc[0]['input_lang']
print(lang)
corpus_df = pd.read_pickle('/home/amey/projects/multilingual-long-context-LLMs/data/distractors/mlqa.pkl')
corpus_df = corpus_df[corpus_df['context_lang'] == lang][:10]
corpus = corpus_df['context'].tolist()

print(source_df.shape, corpus_df.shape)
source_df.head()

en
(10, 7) (10, 3)


,Unnamed: 0,passage,question,question_type,input_lang,target,id
0,0,John travelled to the hallway. Mary journeyed ...,Where is Mary?,qa1,en,bathroom,0_en
1,1,John travelled to the hallway. Mary journeyed ...,Where is Sandra?,qa1,en,kitchen,1_en
2,2,John travelled to the office. Mary journeyed t...,Where is Mary?,qa1,en,kitchen,2_en
3,3,John moved to the hallway. John journeyed to t...,Where is John?,qa1,en,kitchen,3_en
4,4,Sandra moved to the bedroom. Mary went back to...,Where is Sandra?,qa1,en,bedroom,4_en


In [6]:
import torch
from sentence_transformers import util

if torch.cuda.is_available():
    device = 'cuda'
    print("CUDA is available. Using GPU.")
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('/home/models/paraphrase-multilingual-mpnet-base-v2', device=device)
corpus_embeddings = model.encode(corpus, convert_to_tensor=True, device=device)


def find_top_k_similar(passage, k=1000):
    passage_embedding = model.encode(passage, convert_to_tensor=True, device=device)
    similarity_scores = util.cos_sim(passage_embedding, corpus_embeddings)[0]
    top_k_scores, top_k_indices = torch.topk(similarity_scores, k=k)
    top_k_contexts = [corpus[idx] for idx in top_k_indices]
    return top_k_contexts

source_df['context_passages'] = source_df['passage'].progress_apply(find_top_k_similar, k=len(corpus) or 1000)


CUDA is available. Using GPU.


NameError: name 'corpus' is not defined

In [31]:
source_df['context_passages'].iloc[0]

['In January 2006, space historian Dwayne A. Day published an article in online aerospace magazine The Space Review titled "Astronauts and Area 51: the Skylab Incident". The article was based on a memo written in 1974 to CIA director William Colby by an unknown CIA official. The memo reported that astronauts on board Skylab 4 had, as part of a larger program, inadvertently photographed a location of which the memo said:',
 'The lakebed made an ideal strip from which they could test aircraft, and the Emigrant Valley\'s mountain ranges and the NTS perimeter, about 100 mi (160 km) north of Las Vegas, protected the test site from visitors. The CIA asked the AEC to acquire the land, designated "Area 51" on the map, and add it to the Nevada Test Site.Johnson named the area "Paradise Ranch" to encourage workers to move to a place that the CIA\'s official history of the U-2 project would later describe as "the new facility in the middle of nowhere"; the name became shortened to "the Ranch".  O

#### Type 1 - MLQA

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import torch
from sentence_transformers import util

if torch.cuda.is_available():
    device = 'cuda'
    print("CUDA is available. Using GPU.")
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")

from sentence_transformers import SentenceTransformer

def find_top_k_similar(passage, k=1000):
    passage_embedding = model.encode(passage, convert_to_tensor=True, device=device)
    similarity_scores = util.cos_sim(passage_embedding, corpus_embeddings)[0]
    top_k_scores, top_k_indices = torch.topk(similarity_scores, k=k)
    top_k_contexts = [corpus[idx] for idx in top_k_indices]
    return top_k_contexts

corpus_df = pd.read_pickle('/home/amey/projects/multilingual-long-context-LLMs/data/distractors/mlqa.pkl')
model = SentenceTransformer('/home/models/paraphrase-multilingual-mpnet-base-v2', device=device)

for filename in glob.glob('/home/amey/projects/multilingual-long-context-LLMs/data/babilong/multilingual_bAbi/*.csv'):
    # Load your data into DataFrames
    source_df = pd.read_csv(filename)
    lang = source_df.iloc[0]['input_lang']
    corpus = corpus_df[corpus_df['context_lang'] == lang]['context'].tolist()
    corpus_embeddings = model.encode(corpus, convert_to_tensor=True, device=device)
    source_df['distractor_passages_mlqa'] = source_df['passage'].progress_apply(find_top_k_similar, 1000)
    source_df.to_pickle(f'/home/amey/projects/multilingual-long-context-LLMs/data/input/{lang}.pkl')


CUDA is available. Using GPU.


100%|██████████| 1000/1000 [00:32<00:00, 30.88it/s]


#### Type 2 - Synthetic Distractors

In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import torch
from sentence_transformers import util

if torch.cuda.is_available():
    device = 'cuda'
    print("CUDA is available. Using GPU.")
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")

from sentence_transformers import SentenceTransformer

def find_top_k_similar(passage, k=1000):
    passage_embedding = model.encode(passage, convert_to_tensor=True, device=device)
    similarity_scores = util.cos_sim(passage_embedding, corpus_embeddings)[0]
    top_k_scores, top_k_indices = torch.topk(similarity_scores, k=k)
    top_k_contexts = [corpus[idx] for idx in top_k_indices]
    return top_k_contexts



corpus_df = pd.read_pickle('/home/amey/projects/multilingual-long-context-LLMs/data/distractors/synthetic_llama.pkl')
model = SentenceTransformer('/home/models/paraphrase-multilingual-mpnet-base-v2', device=device)

for filename in glob.glob('/home/amey/projects/multilingual-long-context-LLMs/data/input/*.pkl'):
    # Load your data into DataFrames
    source_df = pd.read_pickle(filename)
    lang = source_df.iloc[0]['input_lang']
    corpus = corpus_df[corpus_df['context_lang'] == lang]['context'].tolist()
    corpus_embeddings = model.encode(corpus, convert_to_tensor=True, device=device)
    source_df['distractor_passages_synthetic'] = source_df['passage'].progress_apply(find_top_k_similar, 1000)
    source_df.to_pickle(f'/home/amey/projects/multilingual-long-context-LLMs/data/input/{lang}.pkl')
source_df

CUDA is available. Using GPU.


100%|██████████| 1000/1000 [00:26<00:00, 37.41it/s]


,Unnamed: 0,passage,question,question_type,input_lang,target,id,distractor_passages_mlqa,distractor_passages_synthetic
0,6000,Hans ging zum Flur. Bertha ging ins Badezimmer...,Wo ist Bertha?,qa1,de,Badezimmer,0_de,[26 Jahre später ziehen die Waisen Judy und Pe...,"[Hans ist im Badezimmer, ein Name, der oft mit..."
1,6001,Hans ging zum Flur. Bertha ging ins Badezimmer...,Wo ist Sabine?,qa1,de,Küche,1_de,[Phoebe war vor Beginn der Serie lange Monicas...,"[Hans ist im Badezimmer, ein Name, der oft mit..."
2,6002,Hans reiste ins Büro. Bertha reiste in die Küche.,Wo ist Bertha?,qa1,de,Küche,2_de,"[Die Vermieterin, Mrs. Brooks, lauscht am Schl...","[Bertha ist nicht mehr in der Küche, was bedeu..."
3,6003,Hans ging in den Flur. Hans reiste in die Küch...,Wo ist Hans?,qa1,de,Küche,3_de,[Am nächsten Tag ist das Feuer in der Überwach...,"[Hans ist im Badezimmer, ein Name, der oft mit..."
4,6004,Sabine ging ins Schlafzimmer. Bertha ging zurü...,Wo ist Sabine?,qa1,de,Schlafzimmer,4_de,"[Die Vermieterin, Mrs. Brooks, lauscht am Schl...",[Die Schlafzimmer und Küche sind zwei der häuf...
...,...,...,...,...,...,...,...,...,...
995,6995,Fritz ist entweder in der Küche oder im Büro. ...,Ist Fritz in der Küche?,qa10,de,Vielleicht,995_de,[Zu Beginn des Filmes wohnt der 78-jährige Car...,[Fritz ist entweder im Schlafzimmer oder im Bü...
996,6996,Bertha ist entweder in der Schule oder im Büro...,Ist Julia in der Schule?,qa10,de,Vielleicht,996_de,[Tuesday übernimmt am folgenden Tag die Rolle ...,"[Bertha ist nicht im Flur, was man sich vorste..."
997,6997,Bertha ist entweder in der Schule oder im Büro...,Ist Julia in der Küche?,qa10,de,NEIN,997_de,[Während ihre Schwester ihr aus einem Buch vor...,[Bertha ist entweder in der Schule oder im Par...
998,6998,Wilhelm ging zurück ins Büro. Julia ging in di...,Ist Julia im Schlafzimmer?,qa10,de,Ja,998_de,[26 Jahre später ziehen die Waisen Judy und Pe...,"[Bertha, die berühmte Astronautin, war nicht i..."


### Zero-shot prompts

In [34]:
from prompt_templates import (
    prompt_template_zs_qa1,
    prompt_template_zs_qa2,
    prompt_template_zs_qa3,
    prompt_template_zs_qa4,
    prompt_template_zs_qa5,
    prompt_template_zs_qa6,
    prompt_template_zs_qa7,
    prompt_template_zs_qa8,
    prompt_template_zs_qa9,
    prompt_template_zs_qa10
)



data_dir = '/home/amey/projects/multilingual-long-context-LLMs/data/babilong/multilingual_bAbi'
prompt_dir = '/home/amey/projects/multilingual-long-context-LLMs/prompts/zeroshot'


for filename in glob.glob(f"{data_dir}/*.csv"):
    df = pd.read_csv(filename)
    
    # Take only 10 values per question
    df = df.groupby("question_type").sample(n=10, random_state=42)
    df = df.reset_index(drop=True)

    prompts = []
    for i, row in df.iterrows():
        question_type = row['question_type']
        if question_type == 'qa1':
            prompt = prompt_template_zs_qa1(context=row['passage'], question=row['question'])
        elif question_type == 'qa2':
            prompt = prompt_template_zs_qa2(context=row['passage'], question=row['question'])
        elif question_type == 'qa2':
            prompt = prompt_template_zs_qa2(context=row['passage'], question=row['question'])
        elif question_type == 'qa3':
            prompt = prompt_template_zs_qa3(context=row['passage'], question=row['question'])
        elif question_type == 'qa4':
            prompt = prompt_template_zs_qa4(context=row['passage'], question=row['question'])
        elif question_type == 'qa5':
            prompt = prompt_template_zs_qa5(context=row['passage'], question=row['question'])
        elif question_type == 'qa6':
            prompt = prompt_template_zs_qa6(context=row['passage'], question=row['question'])
        elif question_type == 'qa7':
            prompt = prompt_template_zs_qa7(context=row['passage'], question=row['question'])
        elif question_type == 'qa8':
            prompt = prompt_template_zs_qa8(context=row['passage'], question=row['question'])
        elif question_type == 'qa9':
            prompt = prompt_template_zs_qa9(context=row['passage'], question=row['question'])
        elif question_type == 'qa10':
            prompt = prompt_template_zs_qa10(context=row['passage'], question=row['question'])
        prompts.append(prompt.strip())

    df['prompt'] = prompts
    lang = df.iloc[0].input_lang
    df[['id', 'input_lang', 'prompt', 'target']].to_pickle(f"{prompt_dir}/{lang}.pkl")

,Unnamed: 0,passage,question,question_type,input_lang,target,prompt
0,4083,سافرت ليلى إلى المطبخ. ذهب عمر إلى الحمام. ذهب...,أين عمر؟?,qa1,ar,حمام,"Read the provided text, which contains facts a..."
1,4053,عادت ليلى إلى غرفة النوم. انتقلت مريم إلى المط...,أين يوسف؟?,qa1,ar,حديقة,"Read the provided text, which contains facts a..."
2,4070,سافر عمر إلى الحديقة. انتقل يوسف إلى الحديقة. ...,أين يوسف؟?,qa1,ar,مكتب,"Read the provided text, which contains facts a..."
3,4045,انتقلت ليلى إلى الحديقة. ذهب يوسف إلى غرفة الن...,أين ليلى؟?,qa1,ar,مكتب,"Read the provided text, which contains facts a..."
4,4044,عاد عمر إلى المطبخ. انتقل عمر إلى المكتب. عاد ...,أين عمر؟?,qa1,ar,مكتب,"Read the provided text, which contains facts a..."


### Generate Individual Input files per Lang

In [2]:
import re

def extract_passage_and_question(input_string):
    passage_match = re.search(r'Passage:\s*(.*?)\s*Question:', input_string, re.DOTALL)
    question_match = re.search(r'Question:\s*(.*?)\s*Answer:', input_string, re.DOTALL)
    
    passage = passage_match.group(1).strip() if passage_match else None
    question = question_match.group(1).strip() if question_match else None
    
    return passage, question

# Example usage
input_string = """
Answer the question based on the given passage.

Passage: سافر يوسف إلى الردهة. ذهبت مريم إلى الحمام. عاد عمر إلى الحمام. انتقل يوسف إلى غرفة النوم.
Question: أين مريم؟؟
Answer:
"""

passage, question = extract_passage_and_question(input_string)
print("Passage:", passage)
print("Question:", question)


Passage: سافر يوسف إلى الردهة. ذهبت مريم إلى الحمام. عاد عمر إلى الحمام. انتقل يوسف إلى غرفة النوم.
Question: أين مريم؟؟


In [18]:
df = pd.DataFrame()
df_gt = pd.read_csv('/home/amey/projects/multilingual-long-context-LLMs/data/groundtruth.csv')[:1000]
# df_gt = pd.read_csv('/home/amey/projects/multilingual-long-context-LLMs/data/groundtruth.csv')

for filename in glob.glob('/home/amey/projects/multilingual-long-context-LLMs/prompts/*.pkl'):
    _ = pd.read_pickle(filename)[['prompt']]
    _ = pd.read_pickle(filename)[['prompt']][:1000]
    lang = filename.split('/')[-1].split('.')[0]
    _['input_lang'] = lang
    _['target'] = df_gt['target']
    _["question_type"] = "qa" + ((_.index // 100) + 1).astype(str)
    # _ = _.groupby("question_type").sample(n=10, random_state=42)
    # _ = _.reset_index(drop=True)
    df = pd.concat([df , _])
    
print(df.shape)
df.head(3)

(7000, 4)


,prompt,input_lang,target,question_type
0,Answer the question based on the given passage...,vi,"['bathroom', 'स्नानघर', 'baño', 'Badezimmer', ...",qa1
1,Answer the question based on the given passage...,vi,"['kitchen', 'रसोईघर', 'cocina', 'Küche', 'phòn...",qa1
2,Answer the question based on the given passage...,vi,"['kitchen', 'रसोईघर', 'cocina', 'Küche', 'phòn...",qa1


In [19]:
df['passage'] = df['prompt'].apply(lambda x: extract_passage_and_question(x)[0])
df['question'] = df['prompt'].apply(lambda x: extract_passage_and_question(x)[1].replace("??","?"))
df.head(3)

,prompt,input_lang,target,question_type,passage,question
0,Answer the question based on the given passage...,vi,"['bathroom', 'स्नानघर', 'baño', 'Badezimmer', ...",qa1,Minh đi đến hành lang. Mai đi vào phòng tắm. D...,Mai ở đâu?
1,Answer the question based on the given passage...,vi,"['kitchen', 'रसोईघर', 'cocina', 'Küche', 'phòn...",qa1,Minh đi đến hành lang. Mai đi vào phòng tắm. D...,Lan ở đâu?
2,Answer the question based on the given passage...,vi,"['kitchen', 'रसोईघर', 'cocina', 'Küche', 'phòn...",qa1,Minh đi đến văn phòng. Mai đi vào bếp.,Mai ở đâu?


In [20]:
# df.to_pickle('/home/amey/projects/multilingual-long-context-LLMs/data/input/bAbi_700.pkl')

### Fixing Translation Issues

In [21]:
en_to_de = {
    'Sandra': 'Sabine',
    'Daniel': 'Lukas',
    'John': 'Hans',
    'Jeff': 'Jens',
    'Julie': 'Julia',
    'Mary': 'Bertha',
    'Fred': 'Fritz',
    'Bill': 'Wilhelm'
}

en_to_es = {
    'Sandra': 'Alejandra',
    'Daniel': 'Santiago',
    'John': 'Juan',
    'Jeff': 'Javier',
    'Julie': 'Lucia',
    'Mary': 'María',
    'Fred': 'Federico',
    'Bill': 'Guillermo'
}

en_to_hi = {
    'Sandra': 'Sangeeta',
    'Daniel': 'Dhruv',
    'John': 'Jai',
    'Jeff': 'Jatin',
    'Julie': 'Jyoti',
    'Mary': 'Meera',
    'Fred': 'Farid',
    'Bill': 'Kabir'
}

en_to_zh = {
    'Sandra': 'Meiying',
    'Daniel': 'Minghao',
    'John': 'Jian',
    'Jeff': 'Zhihao',
    'Julie': 'Xiuying',
    'Mary': 'Lihua',
    'Fred': 'Guowei',
    'Bill': 'Weiguo'
}

en_to_vi = {
    'Sandra': 'Lan',
    'Daniel': 'Dương',
    'John': 'Minh',
    'Jeff': 'Phúc',
    'Julie': 'Thảo',
    'Mary': 'Mai',
    'Fred': 'Hùng',
    'Bill': 'Bình'
}

en_to_ar = {
    'Sandra': 'Layla',
    'Daniel': 'Omar',
    'John': 'Youssef',
    'Jeff': 'Fadi',
    'Julie': 'Noor',
    'Mary': 'Mariam',
    'Fred': 'Faris',
    'Bill': 'Bilal'
}

def replace_names(text, lang='hi'):
  if lang == 'ar':
    for key, value in en_to_ar.items():
      text = text.replace(key, value)
    return text
  elif lang == 'hi':
    for key, value in en_to_hi.items():
      text = text.replace(key, value)
    return text
  elif lang == 'de':
    for key, value in en_to_de.items():
      text = text.replace(key, value)
    return text
  elif lang == 'es':
    for key, value in en_to_es.items():
      text = text.replace(key, value)
    return text
  elif lang == 'zh':
    for key, value in en_to_zh.items():
      text = text.replace(key, value)
    return text
  elif lang == 'vi':
    for key, value in en_to_vi.items():
      text = text.replace(key, value)
    return text
  else:
    return text

In [22]:
df['target'] = df['target'].apply(lambda x: eval(x)[0])
df['target'].value_counts()

target
no                1057
yes                952
bedroom            567
kitchen            497
bathroom           490
garden             441
office             427
hallway            378
one                350
none               322
nothing            238
apple              238
milk               217
football           189
Mary               133
Jeff               119
Fred               119
Bill               105
maybe               91
two                 28
apple,football      14
football,apple      14
football,milk        7
milk,apple           7
Name: count, dtype: int64

In [23]:
df['target_'] = df['target']

In [24]:
df = df.reset_index()
for i, row in df.iterrows():
    lang = df.iloc[i]['input_lang']
    df.at[i, 'target_'] = replace_names(df.iloc[i]['target_'], lang=lang)        

df.head(1)

,index,prompt,input_lang,target,question_type,passage,question,target_
0,0,Answer the question based on the given passage...,vi,bathroom,qa1,Minh đi đến hành lang. Mai đi vào phòng tắm. D...,Mai ở đâu?,bathroom


In [25]:
df['target_'].value_counts()

target_
no                1057
yes                952
bedroom            567
kitchen            497
bathroom           490
garden             441
office             427
hallway            378
one                350
none               322
nothing            238
apple              238
milk               217
football           189
maybe               91
two                 28
Mary                19
Bertha              19
Mariam              19
Mai                 19
Meera               19
María               19
Lihua               19
Phúc                17
Fritz               17
Hùng                17
Fadi                17
Faris               17
Farid               17
Jatin               17
Javier              17
Federico            17
Guowei              17
Jens                17
Fred                17
Jeff                17
Zhihao              17
Bình                15
Kabir               15
Guillermo           15
Bilal               15
Weiguo              15
Bill                15
Wil

In [26]:
for x in df['target_'].values:
    print(x)

bathroom
kitchen
kitchen
kitchen
bedroom
office
garden
bathroom
kitchen
bedroom
office
office
bathroom
bathroom
bathroom
hallway
kitchen
office
kitchen
bedroom
hallway
bedroom
bedroom
bedroom
bedroom
garden
bedroom
office
office
garden
garden
bathroom
office
kitchen
garden
kitchen
kitchen
garden
garden
bathroom
bedroom
kitchen
office
bedroom
office
office
bathroom
bedroom
office
bedroom
hallway
bedroom
office
garden
kitchen
hallway
hallway
hallway
bedroom
bedroom
bedroom
kitchen
bedroom
garden
bathroom
garden
office
bedroom
garden
bathroom
office
kitchen
bedroom
office
bathroom
office
hallway
office
office
bathroom
hallway
hallway
hallway
bathroom
bathroom
bathroom
garden
bedroom
office
garden
office
bedroom
hallway
kitchen
bedroom
hallway
garden
bedroom
office
garden
kitchen
garden
bedroom
bedroom
kitchen
hallway
kitchen
garden
bedroom
bathroom
bathroom
bedroom
bathroom
kitchen
bedroom
garden
bedroom
bedroom
garden
garden
office
bedroom
garden
bathroom
office
garden
garden
bedroom
hal

In [28]:
for x in df.groupby('input_lang'):
    print(x[1].shape)
    x[1]['target'] = x[1]['target_']
    x[1][['passage', 'question', 'question_type', 'input_lang', 'target']].to_csv(f'/home/amey/projects/multilingual-long-context-LLMs/data/babilong/language_specific_files/{x[0]}.csv')

(1000, 8)
(1000, 8)
(1000, 8)
(1000, 8)
(1000, 8)
(1000, 8)
(1000, 8)


### Synthetic Distractor Passages

In [28]:
import glob

df = pd.DataFrame()

for filename in glob.glob('/home/amey/projects/multilingual-long-context-LLMs/distractor_prompts/*.pkl'):
    _ = pd.read_pickle(filename)
    df = pd.concat([df, _])
    
print(df.shape)

(25300, 11)


In [30]:
df.shape

(25300, 11)

In [31]:
sample = df.iloc[3001]
print(sample.prompt)
print(dashed_line)

You are given an independent sentence from the bAbI dataset, which follows the Subject -> Action -> Object structure.

### Sentence:
Das Badezimmer liegt südlich des Flurs.

### Task:
Generate a **moderate-length passage (100-200 words)** that describes the object but never mentions the subject or the action.

### Constraints:
- The object may appear, but it **must be used in a completely different meaning or context** than in the original sentence.
- The passage must contain **multiple sentences that expand on the topic with background information, examples, or applications**.
- The passage must have **no semantic or narrative connection** to the original sentence.
- The passage must be **complex enough to act as a distractor** when answering a question about the original sentence.
- The passage could include any topic such as science, sports, economics, politics, medicine, philosophy, history, or technology.
- The passage must be in the same language as the input sentence.

Strictly 

In [32]:
print(sample['predictions_/home/models/Qwen2-7B-Instruct_vllm_temperature_1.0'])

Delving into the intricacies of quantum computing reveals a domain where information processing is reimagined on a molecular scale. Actors on this stage are not the traditional silicon transistors but the boundless potential of electrons to exist in multiple states simultaneously, thanks to a principle known as superposition. In the quest for healing, quantum physics has become a beacon of hope, promising cures for previously intractable diseases. The quantum entanglement, demonstrating the eerily instantaneous connection between particles, regardless of distance, could also reshape our understanding of the universe, challenging the very fabric of reality with its predictions.

Moreover, in the near future, these quantum innovations might redefine the role of technology in economics, by potentially increasing computational speed to a hitherto unfathomable degree. The rapid advancements in quantum computing technology have made it a subject of intense interest for superpowers, aiming no

In [33]:
print(sample['predictions_/home/models/Meta-Llama-3.2-3B-Instruct_vllm'])

Das Badezimmer, ein Ort, der oft mit Erinnerungen und persönlichen Gesetzen assoziiert wird, ist in der Regel ein kleiner, aber wichtiger Teil eines Hauses. Es dient als Ort, um sich nach einem langen Tag zu entspannen, sich zu waschen und sich frisch zu machen. In vielen Kulturen werden Badezimmer als wichtige Bereiche für die persönliche Hygiene und das Wohlbefinden betrachtet. Im Gegensatz dazu ist die Architektur eines Badezimmers oft minimalistisch und funktional gestaltet, um den Fokus auf die Bedürfnisse des Einzelnen zu legen. In einigen Ländern werden Badezimmer auch als wichtige Bereiche für die Gesundheit und das Wohlbefinden betrachtet, da sie Zugang zu sauberem Wasser und sanitären Einrichtungen bieten. In der Medizin wird das Badezimmer oft als Ort betrachtet, an dem Patienten ihre medizinischen Behandlungen durchführen, um ihre Gesundheit zu verbessern.


In [34]:
distractors = []

cols = [col for col in df.columns if 'predictions_' in col and 'Llama-3.2' in col]
for col in cols:
    distractors.extend(df[col].values.tolist())
    
    
distractors[1]

'苹果在中国的历史上是非常重要的食物来源之一，早在汉代就被用作药物和饮料。然而，随着时间的推移，人们对苹果的需求量逐渐减少，直到19世纪中期，苹果在中国的产量才开始迅速增加。今天，苹果不仅仅是食用，也被用于制造各种产品，如果汁、果酱和果醋。苹果还被用于医学研究，例如，研究表明苹果中的抗氧化剂有助于预防心血管疾病。'

In [35]:
len(distractors)

50600

In [36]:
distractors[1]

'苹果在中国的历史上是非常重要的食物来源之一，早在汉代就被用作药物和饮料。然而，随着时间的推移，人们对苹果的需求量逐渐减少，直到19世纪中期，苹果在中国的产量才开始迅速增加。今天，苹果不仅仅是食用，也被用于制造各种产品，如果汁、果酱和果醋。苹果还被用于医学研究，例如，研究表明苹果中的抗氧化剂有助于预防心血管疾病。'

In [41]:
df_distractors = pd.DataFrame()
df_distractors['distractor_passage'] = distractors
df_distractors['id'] = range(len(df_distractors))
df_distractors.head()

,distractor_passage,id
0,Quả bóng đá là một công cụ quan trọng trong nh...,0
1,苹果在中国的历史上是非常重要的食物来源之一，早在汉代就被用作药物和饮料。然而，随着时间的推移...,1
2,卫生间是人们日常生活中的一个基本设施，用于清洁身体和整理个人卫生。健康的卫生间不仅仅是为了保...,2
3,厨房里是一种传统的中国家具制造工艺。它源自于中国的传统手工艺，通过手工雕刻和打磨来制成。这种...,3
4,Die Badezimmer sind ein wichtiger Teil eines H...,4


In [42]:
import langdetect

In [43]:
from tqdm import tqdm
tqdm.pandas()

df_distractors['lang'] = df_distractors['distractor_passage'].progress_apply(lambda x: langdetect.detect(x))
df_distractors.head()

100%|██████████| 50600/50600 [04:03<00:00, 207.67it/s]


,distractor_passage,id,lang
0,Quả bóng đá là một công cụ quan trọng trong nh...,0,vi
1,苹果在中国的历史上是非常重要的食物来源之一，早在汉代就被用作药物和饮料。然而，随着时间的推移...,1,zh-cn
2,卫生间是人们日常生活中的一个基本设施，用于清洁身体和整理个人卫生。健康的卫生间不仅仅是为了保...,2,zh-cn
3,厨房里是一种传统的中国家具制造工艺。它源自于中国的传统手工艺，通过手工雕刻和打磨来制成。这种...,3,zh-cn
4,Die Badezimmer sind ein wichtiger Teil eines H...,4,de


In [45]:
map = {
    ''
}

df_distractors['lang_formatted'] = df_distractors['lang'].progress_apply(lambda x: x.replace('zh-cn', 'zh').replace('zh-tw','zh').replace('mr','hi').replace('ko', 'zh'))
df_distractors['lang_formatted'] = df_distractors['lang_formatted'].progress_apply(lambda x: x if x in ['hi', 'zh', 'de', 'es', 'vi', 'ar'] else 'en')
df_distractors['lang_formatted'].value_counts()

100%|██████████| 50600/50600 [00:00<00:00, 1120193.09it/s]


lang_formatted
en    16863
ar     7063
hi     7044
zh     6170
es     5236
vi     4744
de     3480
Name: count, dtype: int64

In [47]:
df_distractors['context_lang'] = df_distractors['lang_formatted']
df_distractors['context'] = df_distractors['distractor_passage']
df_distractors[['id', 'context', 'context_lang']].to_pickle('/home/amey/projects/multilingual-long-context-LLMs/data/distractors/synthetic_llama.pkl')